# Search for Self-Similar Solutions

In [1]:
import sys
sys.path.append('../src')
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import newton
from sklearn.neighbors import KernelDensity
from scipy.io import savemat
from Davidenko import Davidenko
from Diffusion import Diffusion

In [2]:
% matplotlib inline

Start with the heat equation.

Define the initial conditions and objective function.

In [3]:
minpos, maxpos, points, its = -1, 1, 200, 239
def LennardJones(x):
    return (1 - 2 * x ** 6) / x ** 12
def dLennardJones(x):
    return 12 * (x ** 6 -1) / x ** 13
def ddLennardJones(x):
    return (156 - 84 * x ** 6) / x ** 14
def quartic(x):
    return x ** 4 - x ** 2
def dquartic(x):
    return 4 * x ** 3 - 2 * x
def ddquartic(x):
    return 12 * x ** 2 - 2

Define a Newton Solver method to compute the new positions after each iteration.

In [4]:
def NewtonSimilarSolver(x, df, ddf, its):
    positions = np.empty((its + 1, x.shape[0]))
    positions[0, :] = x
    for i in range(its):
        for j in range(positions.shape[1]):
            positions[i+1, j] = newton(df, positions[i, j], ddf, tol=float('Infinity'), maxiter=1)
        positions[i+1, :] -= min(positions[i+1, :])
        positions[i+1, :] *= (max(positions[i, :] - min(positions[i, :]))) / max(positions[i+1, :])
        positions[i+1, :] += min(positions[i, :])
    return positions

In [5]:
positions = NewtonSimilarSolver(np.linspace(minpos, maxpos, points), dquartic, ddquartic, its)

Perform KDE on the positions.

In [6]:
kde = KernelDensity(bandwidth=3 * (maxpos - minpos)/points, kernel='gaussian')
samples = np.linspace(minpos, maxpos, points)
NewtonDensity = np.empty((its + 1, points))
for i in range(its + 1):
    kde.fit(positions[i, :][:, np.newaxis])
    NewtonDensity[i, :] = np.exp(kde.score_samples(samples[:, np.newaxis]))

In [7]:
#fig, ax = plt.subplots(its + 1, 1, figsize=(4, 8))
#for i in range(its + 1):
#    ax[i].plot(samples, density[i, :])
#    if i != its:
#        ax[i].set_xticks([])
#plt.tight_layout()

Define a Davidenko Solver method to compute the new positions after each iteration.

In [8]:
def DavidenkoSimilarSolver(x, df, ddf, its, trange=np.linspace(0, 1)):
    positions = np.empty((its + 1, x.shape[0]))
    positions[0, :] = x
    for i in range(its):
        positions[i+1, :] = Davidenko(positions[i, :], df, ddf, trange=trange).T[-1, :]
        positions[i+1, :] -= min(positions[i+1, :])
        positions[i+1, :] *= (max(positions[i, :] - min(positions[i, :]))) / max(positions[i+1, :])
        positions[i+1, :] += min(positions[i, :])
    return positions

In [9]:
positions = DavidenkoSimilarSolver(np.linspace(minpos, maxpos, points), dLennardJones, ddLennardJones, its)

Perform KDE on the positions.

In [10]:
kde = KernelDensity(bandwidth=3 * (maxpos - minpos)/points, kernel='gaussian')
samples = np.linspace(minpos, maxpos, points)
DavidenkoDensity = np.empty((its + 1, points))
for i in range(its + 1):
    kde.fit(positions[i, :][:, np.newaxis])
    DavidenkoDensity[i, :] = np.exp(kde.score_samples(samples[:, np.newaxis]))

In [11]:
savemat('Self-Similar-Newton', {'NewtonDensity' : NewtonDensity, 'NewtonPositions' : samples,
                                'DavidenkoDensity' : DavidenkoDensity, 'DavidenkoPositions' : samples})